In [1110]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse.linalg as spla
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# from clubs.models import User, Club, Club_Users, Club_Books, Book, User_Books

pd.options.mode.chained_assignment = None  # default='warn'

In [1111]:
# Junction tables must be filled when new user signed up!

book_df = pd.read_csv('./clubs_book.csv', encoding = "Latin1", sep = ",")
user_df = pd.read_csv('./clubs_user.csv', encoding = "Latin1", sep = ",")
club_df = pd.read_csv('./clubs_club.csv', encoding = "Latin1", sep = ",")
club_book_df = pd.read_csv('./clubs_club_books.csv', encoding = "Latin1", sep = ",")
club_user_df = pd.read_csv('./clubs_club_users.csv', encoding = "Latin1", sep = ",")
user_book_df = pd.read_csv('./clubs_user_books.csv', encoding = 'Latin1', sep = ',')

In [1112]:
# Merge club_user junction table with user table to get ages of all users

club_user_age_df = club_user_df.merge(user_df, left_on = 'user_id', right_on = 'id')
club_user_age_df = club_user_age_df[['id_x', 'club_id', 'user_id', 'age']]
club_user_age_df = club_user_age_df.rename(columns={'id_x':'club_user_id'}).sort_values('club_user_id', ascending=True)

club_user_age_df

,club_user_id,club_id,user_id,age
0,1,2,207351,115
1,2,2,241265,136
3,3,1,11909,41
6,4,5,31325,143
7,5,4,249795,114
...,...,...,...,...
12,496,10,72722,57
455,497,8,216249,103
294,498,2,144369,75
187,499,2,185348,83


In [1113]:
# Merge club_user junction table with club table to get locations of all clubs

club_user_location_df = club_user_df.merge(club_df, left_on = 'club_id', right_on = 'id')
club_user_location_df = club_user_location_df[['id_x', 'club_id', 'user_id', 'location']]
club_user_location_df = club_user_location_df.rename(columns={'id_x':'club_user_id'}).sort_values('club_user_id', ascending=True)

club_user_location_df

,club_user_id,club_id,user_id,location
0,1,2,207351,"Tokyo, Japan"
1,2,2,241265,"Tokyo, Japan"
63,3,1,11909,"Adelaide, Australia"
108,4,5,31325,"Sharm El Sheikh, Egypt"
156,5,4,249795,"Luxor, Egypt"
...,...,...,...,...
455,496,10,72722,"London, UK"
360,497,8,216249,"Delhi, India"
60,498,2,144369,"Tokyo, Japan"
61,499,2,185348,"Tokyo, Japan"


In [1114]:
# Merge the club_user_age and club_user_location tables to have all in one table

club_user_age_location_df = club_user_age_df.merge(club_user_location_df, left_on = 'club_user_id', right_on = 'club_user_id')
club_user_age_location_df = club_user_age_location_df[['club_user_id', 'club_id_x', 'user_id_x', 'age', 'location']]

club_user_age_location_df = club_user_age_location_df.rename(columns={'user_id_x':'user_id', 'club_id_x':'club_id'})

club_user_age_location_df

,club_user_id,club_id,user_id,age,location
0,1,2,207351,115,"Tokyo, Japan"
1,2,2,241265,136,"Tokyo, Japan"
2,3,1,11909,41,"Adelaide, Australia"
3,4,5,31325,143,"Sharm El Sheikh, Egypt"
4,5,4,249795,114,"Luxor, Egypt"
...,...,...,...,...,...
495,496,10,72722,57,"London, UK"
496,497,8,216249,103,"Delhi, India"
497,498,2,144369,75,"Tokyo, Japan"
498,499,2,185348,83,"Tokyo, Japan"


In [1115]:
# Get location and average age of each club

average_club_age_df = pd.merge(club_user_age_df, club_df, left_on='club_id', right_on='id') \
    .groupby(['club_id', 'name', 'location'])['age'].mean().reset_index(name = 'average_age')

average_club_age_df

,club_id,name,location,average_age
0,1,SarahMatthews's Club,"Adelaide, Australia",62.400000
1,2,HelenGray's Club,"Tokyo, Japan",69.857143
2,3,RhysNicholson's Club,"Sao Paulo, Brazil",66.660377
3,4,KarenQuinn's Club,"Luxor, Egypt",61.720000
4,5,CameronHarvey's Club,"Sharm El Sheikh, Egypt",80.979167
5,6,ArthurShepherd's Club,"Ottawa, Canada",65.490909
6,7,FrancesPhillips's Club,"Belo Horizonte, Brazil",77.886364
7,8,SylviaFranklin's Club,"Delhi, India",87.106383
8,9,GeorgeBryan's Club,"Ibadan, Nigeria",75.925000
9,10,CarolKing's Club,"London, UK",68.672727


In [1116]:
# Add column for age difference and return clubs in ascending order of difference from my age

my_age = 65
average_club_age_df['age_difference'] = pd.DataFrame(abs(average_club_age_df['average_age'] - my_age).apply(np.floor))
average_club_age_difference_df = average_club_age_df.sort_values('age_difference', ascending=True)
average_club_age_difference_df

,club_id,name,location,average_age,age_difference
5,6,ArthurShepherd's Club,"Ottawa, Canada",65.490909,0.0
2,3,RhysNicholson's Club,"Sao Paulo, Brazil",66.660377,1.0
0,1,SarahMatthews's Club,"Adelaide, Australia",62.400000,2.0
3,4,KarenQuinn's Club,"Luxor, Egypt",61.720000,3.0
9,10,CarolKing's Club,"London, UK",68.672727,3.0
1,2,HelenGray's Club,"Tokyo, Japan",69.857143,4.0
8,9,GeorgeBryan's Club,"Ibadan, Nigeria",75.925000,10.0
6,7,FrancesPhillips's Club,"Belo Horizonte, Brazil",77.886364,12.0
4,5,CameronHarvey's Club,"Sharm El Sheikh, Egypt",80.979167,15.0
7,8,SylviaFranklin's Club,"Delhi, India",87.106383,22.0


In [1117]:
# Return top 10 closest aged club IDs

closest_age_clubs_df = average_club_age_difference_df['club_id'].iloc[0:5]
closest_age_clubs_df

5     6
2     3
0     1
3     4
9    10
Name: club_id, dtype: int64

In [1118]:
# Merge closest aged club IDs with clubs CSV to get all details
closest_age_clubs_df = closest_age_clubs_df.reset_index().rename(columns={'club_id':'id'})
closest_age_clubs_df = pd.merge(closest_age_clubs_df, club_df, on = 'id')

closest_age_clubs_df

,index,id,name,location,description,avg_reading_speed,owner_id
0,5,6,ArthurShepherd's Club,"Ottawa, Canada",Eius labore dicta quaerat.\r\nAssumenda necess...,316,78
1,2,3,RhysNicholson's Club,"Sao Paulo, Brazil",Commodi a debitis beatae repellat sunt nesciun...,79,68227
2,0,1,SarahMatthews's Club,"Adelaide, Australia",Iusto repellat inventore nihil reiciendis. Dis...,364,271772
3,3,4,KarenQuinn's Club,"Luxor, Egypt",Sed dolorum in totam ad dolore. Magnam cum ea ...,384,209308
4,9,10,CarolKing's Club,"London, UK",Repellat iure sed reprehenderit amet similique...,158,10319


In [1119]:
# Get user count of each club

club_user_count_df = pd.merge(club_user_age_df, club_df, left_on='club_id', right_on='id') \
    .groupby(['club_id', 'name'])['user_id'].count().reset_index(name = 'user_count')

club_user_count_df

,club_id,name,user_count
0,1,SarahMatthews's Club,45
1,2,HelenGray's Club,63
2,3,RhysNicholson's Club,53
3,4,KarenQuinn's Club,50
4,5,CameronHarvey's Club,48
5,6,ArthurShepherd's Club,55
6,7,FrancesPhillips's Club,44
7,8,SylviaFranklin's Club,47
8,9,GeorgeBryan's Club,40
9,10,CarolKing's Club,55


In [1120]:
# Return clubs with matching location (exact)

my_location = 'Adelaide, Australia'

location_match = club_df['location'] == my_location
closest_location_clubs_df = club_df[location_match]

closest_location_clubs_df

,id,name,location,description,avg_reading_speed,owner_id
0,1,SarahMatthews's Club,"Adelaide, Australia",Iusto repellat inventore nihil reiciendis. Dis...,364,271772


In [1121]:
# Return all clubs with boolean value for matching location (using fuzzy search)

# Set test location for user
user_location = "Tokyo, Japan"
user_id = 25765


club_user_location_df['location_match_score'] = np.nan
club_user_location_df

no_of_rows = club_user_location_df.shape[0]

for i in range(no_of_rows):
    match_value = int(fuzz.token_sort_ratio(user_location, club_user_location_df.iloc[i]['location']))
    if (user_id != club_user_location_df.iloc[i]['user_id']):
        club_user_location_df.iat[i,4] = match_value
    else:
        club_user_age_location_df.drop(i)

club_user_location_df = club_user_location_df.sort_values('location_match_score', ascending=False).dropna(how='any',axis=0)
club_user_location_df = club_user_location_df.drop('club_user_id', axis = 1).groupby(['club_id', 'location', 'location_match_score']).agg(list).reset_index()
club_user_location_df = club_user_location_df[club_user_location_df['location_match_score'] > 80]
club_user_location_df
# if club_user_location_df.empty:
#     print("No matching clubs found")
# else:
#     print(club_user_location_df) #make it a return statement


,club_id,location,location_match_score,user_id
1,2,"Tokyo, Japan",100.0,"[207351, 272312, 176059, 215620, 72779, 98777,..."


In [1122]:
# Perform a many-to-many merge to get the favourite books of each club

club_favourite_books_df = pd.merge(pd.merge(club_df, club_book_df, left_on='id', right_on='club_id'), 
                    pd.merge(book_df, club_book_df, left_on='id', right_on='book_id'), on='book_id', how = 'inner') \
                        .groupby(['club_id_x', 'name', 'ISBN'])['title', 'author'].agg(list).reset_index()

club_favourite_books_df = club_favourite_books_df.rename(columns={'club_id_x':'club_id'})

club_favourite_books_df['title'] = club_favourite_books_df['title'].str[0]
club_favourite_books_df['author'] = club_favourite_books_df['author'].str[0]

club_favourite_books_df

/var/folders/n7/7tl74pbd6m54b0r639d_dy880000gn/T/ipykernel_92199/1279532313.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  club_favourite_books_df = pd.merge(pd.merge(club_df, club_book_df, left_on='id', right_on='club_id'),


,club_id,name,ISBN,title,author
0,1,SarahMatthews's Club,0140016929,From London Far,Michael Innes
1,1,SarahMatthews's Club,0671685244,DEAD ON TARGET (HB #1) (Hardy Boys Casefiles (...,Franklin W. Dixon
2,1,SarahMatthews's Club,0688026826,Shadow Magic,Seymour Simon
3,1,SarahMatthews's Club,0785809880,The Pre-Raphaelites (Centuries of Style),Inc. Book Sales
4,1,SarahMatthews's Club,0812565665,A Cure for Gravity,Arthur Rosenfeld
5,2,HelenGray's Club,0023376627,The Conscious Reader,Caroline Schrodes
6,2,HelenGray's Club,0192816640,The Expedition of Humphry Clinker (The World's...,Tobias Smollett
7,2,HelenGray's Club,0198319746,Twelfth Night (Oxford School Shakespeare Series),Roma Gill
8,2,HelenGray's Club,0764507508,"Digital Photography for Dummies, Quick Reference",David D. Busch
9,2,HelenGray's Club,0965064573,Ferocious Romance: What My Encounters With Th...,Donna Minkowitz


In [1123]:
# Perform a many-to-many merge to get the favourite books of each user

user_favourite_books_df = pd.merge(pd.merge(user_df, user_book_df, left_on='id', right_on='user_id'), 
                    pd.merge(book_df, user_book_df, left_on='id', right_on='book_id'), on='book_id', how = 'inner') \
                       .groupby(['user_id_x', 'first_name','last_name', 'ISBN', 'title', 'author'])['title', 'author'].agg(list).reset_index()

user_favourite_books_df = user_favourite_books_df.rename(columns={'user_id_x':'user_id'}).drop(0, 1)
user_favourite_books_df
user_club_favourite_books_df = pd.merge(user_favourite_books_df, club_user_df, left_on="user_id", right_on="user_id", how="inner")
user_club_favourite_books_df = user_club_favourite_books_df[['club_id', 'user_id', 'title', 'author']]
user_club_favourite_books_df = user_club_favourite_books_df.sort_values('user_id', ascending=True)
user_club_favourite_books_df = user_club_favourite_books_df.groupby(['user_id', 'title', 'author']).agg(list).reset_index()
user_club_favourite_books_df

/var/folders/n7/7tl74pbd6m54b0r639d_dy880000gn/T/ipykernel_92199/2196139791.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  user_favourite_books_df = pd.merge(pd.merge(user_df, user_book_df, left_on='id', right_on='user_id'),
/var/folders/n7/7tl74pbd6m54b0r639d_dy880000gn/T/ipykernel_92199/2196139791.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_favourite_books_df = user_favourite_books_df.rename(columns={'user_id_x':'user_id'}).drop(0, 1)


,user_id,title,author,club_id
0,78,Sisters in Crime 3 (Sisters in Crime),Marilyn Wallace,[8]
1,78,Surfside High: Sandy (Surfside High),Virginia Baxter,[8]
2,78,The Pilgrims' Progress,John Bunuan,[8]
3,78,To Marry McCloud (Bachelor Cousins) (Harlequi...,Carole Mortimer,[8]
4,78,Traum im Herbst. Und andere StÃ?ÃÂ¼cke.,Jon Fosse,[8]
...,...,...,...,...
2476,276432,A Redoute Treasury: 468 Watercolours from Les ...,Peter Mallary,[9]
2477,276432,Midnight Sun,Kat Martin,[9]
2478,276432,Overcoming Writing Blocks,"Karin and Skjei, Eric MacK",[9]
2479,276432,Shadow Magic,Seymour Simon,[9]


In [1124]:
# Get the favourite books and authors of one user (me)

my_id = 142448
my_favourite_books_df = user_club_favourite_books_df.loc[user_club_favourite_books_df['user_id'] == my_id]
my_favourite_books_df

,user_id,title,author,club_id
1324,142448,Running on Empty: Refilling Your Spirit at the...,Jill Briscoe,[6]
1325,142448,The Meaning of Consuelo : A Novel,Judith Ortiz Cofer,[6]
1326,142448,The Pre-Raphaelites (Centuries of Style),Inc. Book Sales,[6]
1327,142448,Ungrateful Dead,Gary L. Holleman,[6]
1328,142448,"Was ist was?, Bd.32, Meereskunde",Rainer Crummenerl,[6]


In [1125]:
# Get the favourite books and authors of one club

club_id = 1
single_club_favourite_books_df = club_favourite_books_df.loc[club_favourite_books_df['club_id'] == club_id]
single_club_favourite_books_df

,club_id,name,ISBN,title,author
0,1,SarahMatthews's Club,0140016929,From London Far,Michael Innes
1,1,SarahMatthews's Club,0671685244,DEAD ON TARGET (HB #1) (Hardy Boys Casefiles (...,Franklin W. Dixon
2,1,SarahMatthews's Club,0688026826,Shadow Magic,Seymour Simon
3,1,SarahMatthews's Club,0785809880,The Pre-Raphaelites (Centuries of Style),Inc. Book Sales
4,1,SarahMatthews's Club,0812565665,A Cure for Gravity,Arthur Rosenfeld


In [1126]:
# Perform a many-to-many merge to get the favourite books of each user

user_favourite_books_df = pd.merge(pd.merge(user_df, user_book_df, left_on='id', right_on='user_id'), 
                    pd.merge(book_df, user_book_df, left_on='id', right_on='book_id'), on='book_id', how = 'inner') \
                       .groupby(['user_id_x', 'first_name','last_name', 'ISBN', 'title', 'author'])['title', 'author'].agg(list).reset_index()

user_favourite_books_df = user_favourite_books_df.rename(columns={'user_id_x':'user_id'}).drop(0, 1)
user_favourite_books_df
user_club_favourite_books_df = pd.merge(user_favourite_books_df, club_user_df, left_on="user_id", right_on="user_id", how="inner")
user_club_favourite_books_df = user_club_favourite_books_df[['club_id', 'user_id', 'title', 'author']]
user_club_favourite_books_df = user_club_favourite_books_df.sort_values('user_id', ascending=True)
user_club_favourite_books_df = user_club_favourite_books_df.groupby(['user_id', 'title', 'author']).agg(list).reset_index()
user_club_favourite_books_df

/var/folders/n7/7tl74pbd6m54b0r639d_dy880000gn/T/ipykernel_92199/2196139791.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  user_favourite_books_df = pd.merge(pd.merge(user_df, user_book_df, left_on='id', right_on='user_id'),
/var/folders/n7/7tl74pbd6m54b0r639d_dy880000gn/T/ipykernel_92199/2196139791.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  user_favourite_books_df = user_favourite_books_df.rename(columns={'user_id_x':'user_id'}).drop(0, 1)


,user_id,title,author,club_id
0,78,Sisters in Crime 3 (Sisters in Crime),Marilyn Wallace,[8]
1,78,Surfside High: Sandy (Surfside High),Virginia Baxter,[8]
2,78,The Pilgrims' Progress,John Bunuan,[8]
3,78,To Marry McCloud (Bachelor Cousins) (Harlequi...,Carole Mortimer,[8]
4,78,Traum im Herbst. Und andere StÃ?ÃÂ¼cke.,Jon Fosse,[8]
...,...,...,...,...
2476,276432,A Redoute Treasury: 468 Watercolours from Les ...,Peter Mallary,[9]
2477,276432,Midnight Sun,Kat Martin,[9]
2478,276432,Overcoming Writing Blocks,"Karin and Skjei, Eric MacK",[9]
2479,276432,Shadow Magic,Seymour Simon,[9]


In [1127]:
club_user_favourite_books_df = pd.merge(club_favourite_books_df, club_user_df, left_on='club_id', right_on='club_id', how='inner')
club_user_favourite_books_df = club_user_favourite_books_df.drop('id', axis=1).drop('role_num', axis=1)
club_user_favourite_books_df = club_user_favourite_books_df.groupby(['club_id', 'user_id', 'title', 'name', 'ISBN']).agg(list).reset_index()
club_user_favourite_books_df

,club_id,user_id,title,name,ISBN,author
0,1,9274,A Cure for Gravity,SarahMatthews's Club,0812565665,[Arthur Rosenfeld]
1,1,9274,DEAD ON TARGET (HB #1) (Hardy Boys Casefiles (...,SarahMatthews's Club,0671685244,[Franklin W. Dixon]
2,1,9274,From London Far,SarahMatthews's Club,0140016929,[Michael Innes]
3,1,9274,Shadow Magic,SarahMatthews's Club,0688026826,[Seymour Simon]
4,1,9274,The Pre-Raphaelites (Centuries of Style),SarahMatthews's Club,0785809880,[Inc. Book Sales]
...,...,...,...,...,...,...
2410,10,274758,"E-Therapy: Case Studies, Guiding Principles, a...",CarolKing's Club,0393703703,[Robert C. Hsiung]
2411,10,274758,Expedition Whydah: The Story of the World's Fi...,CarolKing's Club,0060192321,[Barry Clifford]
2412,10,274758,Midnight Sun,CarolKing's Club,0821773801,[Kat Martin]
2413,10,274758,The Diary of a Nobody (Essential.penguin S.),CarolKing's Club,0140285563,[George Grossmith]


In [1128]:
# Return all matching favourite books between a single user and multiple clubs (using fuzzy search)

user_id = 25765

no_of_user_favourite_books = my_favourite_books_df.shape[0]
no_of_club_favourite_books = club_user_favourite_books_df.shape[0]
club_user_title_matches_df = club_user_favourite_books_df

club_user_title_matches_df['title_match_score'] = np.nan
club_user_title_matches_df

for i in range(no_of_user_favourite_books):
    my_favourite_book = my_favourite_books_df.iloc[i]

    for j in range(no_of_club_favourite_books):
        club_favourite_book = club_user_title_matches_df.iloc[j]

        match_value = int(fuzz.token_sort_ratio(my_favourite_book['title'], club_favourite_book['title']))
        club_user_title_matches_df.iat[j,6] = match_value

        if (user_id != club_user_title_matches_df.iloc[j]['user_id']):
            club_user_title_matches_df.iat[j,6] = match_value
        else:
            club_user_title_matches_df.drop(j)


club_average_title_match_df = club_user_title_matches_df[['club_id', 'title_match_score', 'title', 'name', 'ISBN']]
club_user_title_matches_df = club_user_title_matches_df.drop('user_id',axis=1)
club_user_title_matches_df = club_user_title_matches_df.groupby(['club_id', 'title_match_score', 'title', 'name', 'ISBN']).agg(list).reset_index()
club_user_title_matches_df = club_user_title_matches_df.sort_values('title_match_score', ascending=False).dropna(how='any',axis=0)
club_user_title_matches_df = club_user_title_matches_df[club_user_title_matches_df['title_match_score'] > 40]

club_user_title_matches_df

# if club_user_title_matches_df.empty:
#     print("No matching clubs found")
# else:
#     print(club_user_title_matches_df) #make it a return statement


,club_id,title_match_score,title,name,ISBN,author
19,4,44.0,"Wolf's Bane (Destroyer Series, No. 132)",KarenQuinn's Club,0373632479,"[[Warren Murphy], [Warren Murphy], [Warren Mur..."


In [1129]:
# Return a list of clubs in order of which have the most matching favourite books with the user

club_average_book_match_df = club_user_title_matches_df.groupby(['club_id', 'name', 'title_match_score'])['title_match_score'] \
    .count().reset_index(name = 'book_match_count') \
    .sort_values('book_match_count', ascending=False) \
    .rename(columns={'name':'club_book_name'})

club_average_book_match_df

,club_id,club_book_name,title_match_score,book_match_count
0,4,KarenQuinn's Club,44.0,1


In [1130]:
club_user_favourite_books_df_2 = pd.merge(club_favourite_books_df, club_user_df, left_on='club_id', right_on='club_id', how='inner')
club_user_favourite_books_df_2 = club_user_favourite_books_df_2.drop('id', axis=1).drop('role_num', axis=1)
club_user_favourite_books_df_2 = club_user_favourite_books_df_2.groupby(['club_id', 'user_id', 'author', 'name', 'ISBN']).agg(list).reset_index()
club_user_favourite_books_df_2

,club_id,user_id,author,name,ISBN,title
0,1,9274,Arthur Rosenfeld,SarahMatthews's Club,0812565665,[A Cure for Gravity]
1,1,9274,Franklin W. Dixon,SarahMatthews's Club,0671685244,[DEAD ON TARGET (HB #1) (Hardy Boys Casefiles ...
2,1,9274,Inc. Book Sales,SarahMatthews's Club,0785809880,[The Pre-Raphaelites (Centuries of Style)]
3,1,9274,Michael Innes,SarahMatthews's Club,0140016929,[From London Far]
4,1,9274,Seymour Simon,SarahMatthews's Club,0688026826,[Shadow Magic]
...,...,...,...,...,...,...
2410,10,274758,Barry Clifford,CarolKing's Club,0060192321,[Expedition Whydah: The Story of the World's F...
2411,10,274758,George Grossmith,CarolKing's Club,0140285563,[The Diary of a Nobody (Essential.penguin S.)]
2412,10,274758,Jeffrey Toobin,CarolKing's Club,0679441700,[The Run of His Life: The People V. O.J. Simpson]
2413,10,274758,Kat Martin,CarolKing's Club,0821773801,[Midnight Sun]


In [1131]:
# Return all matching favourite authors between a single user and multiple clubs (using fuzzy search)
# This works by checking the authors of all of a club's favourite books agains the authors of all of a user's favourite books

user_id = 25765

no_of_user_favourite_books = my_favourite_books_df.shape[0]
no_of_club_favourite_books = club_user_favourite_books_df_2.shape[0]
club_user_author_matches_df = club_user_favourite_books_df_2

club_user_author_matches_df['author_match_score'] = np.nan

for i in range(no_of_user_favourite_books):
    my_favourite_book = my_favourite_books_df.iloc[i]
    

    for j in range(no_of_club_favourite_books):
        club_favourite_book = club_user_author_matches_df.iloc[j]
        
        match_value = int(fuzz.token_sort_ratio(my_favourite_book['author'], club_favourite_book['author']))
        # match_value
        club_user_author_matches_df.iat[j,6] = match_value


        # if (user_id != club_user_author_matches_df.iloc[j]['user_id']):
        #     club_user_author_matches_df.iat[j,7] = match_value
        # else:
        #     club_user_author_matches_df.drop(j)


club_average_author_match_df = club_user_author_matches_df[['club_id', 'author_match_score', 'author', 'name', 'ISBN']]
club_user_author_matches_df = club_user_author_matches_df.drop('user_id',axis=1)
club_user_author_matches_df = club_user_author_matches_df.groupby(['club_id', 'author_match_score', 'author', 'name', 'ISBN']).agg(list).reset_index()
club_user_author_matches_df = club_user_author_matches_df.sort_values('author_match_score', ascending=False).dropna(how='any',axis=0)
club_user_author_matches_df = club_user_author_matches_df[club_user_author_matches_df['author_match_score'] > 40]

club_user_author_matches_df

# if club_user_author_matches_df.empty:
#     print("No matching clubs found")
# else:
#     print(club_user_author_matches_df) #make it a return statement

,club_id,author_match_score,author,name,ISBN,title
39,8,50.0,Elaine Rome,SylviaFranklin's Club,0373286856,"[[Stark Lightning (Harlequin Historical, No. 8..."
29,6,48.0,Merline Lovelace,ArthurShepherd's Club,0373273495,[[The Right Stuff (Silhouette Intimate Moments...
14,3,47.0,Karen MacNeil,RhysNicholson's Club,1563054345,"[[The Wine Bible], [The Wine Bible], [The Wine..."
19,4,44.0,Frank Baer,KarenQuinn's Club,843501665X,"[[El Puente de Alcantara], [El Puente de Alcan..."
9,2,41.0,Caroline Schrodes,HelenGray's Club,0023376627,"[[The Conscious Reader], [The Conscious Reader..."
34,7,41.0,Laurie Paige,FrancesPhillips's Club,0373053045,"[[Misty Splendor (Silhouette Desire, No 304)],..."


In [1132]:
# Return a list of clubs in order of which have the most matching favourite authors with the user

club_average_author_match_df = club_user_author_matches_df.groupby(['club_id', 'name', 'author_match_score'])['author_match_score'] \
    .count().reset_index(name = 'book_match_count') \
    .sort_values('book_match_count', ascending=False) \
    .rename(columns={'name':'club_book_name'})
    
club_average_author_match_df

,club_id,club_book_name,author_match_score,book_match_count
0,2,HelenGray's Club,41.0,1
1,3,RhysNicholson's Club,47.0,1
2,4,KarenQuinn's Club,44.0,1
3,6,ArthurShepherd's Club,48.0,1
4,7,FrancesPhillips's Club,41.0,1
5,8,SylviaFranklin's Club,50.0,1


In [1133]:
# Get all columns into one dataframe

# Location: average_club_age_difference_df
# Age difference: average_club_age_difference_df
# User count: club_user_count_df
# Favourite books: club_average_book_match_df
# Favourite authors: club_average_author_match_df

best_clubs_df = club_user_count_df.merge(average_club_age_difference_df, how = 'left', left_on = 'club_id', right_on = 'club_id')
best_clubs_df = best_clubs_df.merge(club_user_location_df, how = 'left', left_on = 'club_id', right_on = 'club_id')
best_clubs_df = best_clubs_df.merge(club_average_book_match_df, how = 'left', left_on = 'club_id', right_on = 'club_id')
best_clubs_df = best_clubs_df.merge(club_average_author_match_df, how = 'left', left_on = 'club_id', right_on = 'club_id')


best_clubs_df = best_clubs_df[['club_id', 'name_x', 'location_match_score', 'title_match_score', 'author_match_score', 'age_difference', 'user_count', 'book_match_count_x', 'book_match_count_y']]
best_clubs_df = best_clubs_df.rename(columns={'name_x':'club_name', 'book_match_count_x':'title_match_count', 'book_match_count_y':'author_match_count'})

best_clubs_df['title_match_count'] = best_clubs_df['title_match_count'].fillna(0)
best_clubs_df['author_match_count'] = best_clubs_df['author_match_count'].fillna(0)
best_clubs_df['location_match_score'] = best_clubs_df['location_match_score'].fillna(0)
best_clubs_df['title_match_score'] = best_clubs_df['title_match_score'].fillna(0)
best_clubs_df['author_match_score'] = best_clubs_df['author_match_score'].fillna(0)
best_clubs_df


,club_id,club_name,location_match_score,title_match_score,author_match_score,age_difference,user_count,title_match_count,author_match_count
0,1,SarahMatthews's Club,0.0,0.0,0.0,2.0,45,0.0,0.0
1,2,HelenGray's Club,100.0,0.0,41.0,4.0,63,0.0,1.0
2,3,RhysNicholson's Club,0.0,0.0,47.0,1.0,53,0.0,1.0
3,4,KarenQuinn's Club,0.0,44.0,44.0,3.0,50,1.0,1.0
4,5,CameronHarvey's Club,0.0,0.0,0.0,15.0,48,0.0,0.0
5,6,ArthurShepherd's Club,0.0,0.0,48.0,0.0,55,0.0,1.0
6,7,FrancesPhillips's Club,0.0,0.0,41.0,12.0,44,0.0,1.0
7,8,SylviaFranklin's Club,0.0,0.0,50.0,22.0,47,0.0,1.0
8,9,GeorgeBryan's Club,0.0,0.0,0.0,10.0,40,0.0,0.0
9,10,CarolKing's Club,0.0,0.0,0.0,3.0,55,0.0,0.0


In [1134]:
# Order if location matters

# TO DO: Find a way to convert location to distance

best_clubs_df = best_clubs_df.sort_values(by=['location_match_score', 'title_match_score', 'author_match_score', 'title_match_count', 'age_difference', 'author_match_count', 'user_count'], \
    axis=0, ascending = [False, False, False, False, False, True, False], kind='quicksort')
best_clubs_df

,club_id,club_name,location_match_score,title_match_score,author_match_score,age_difference,user_count,title_match_count,author_match_count
1,2,HelenGray's Club,100.0,0.0,41.0,4.0,63,0.0,1.0
3,4,KarenQuinn's Club,0.0,44.0,44.0,3.0,50,1.0,1.0
7,8,SylviaFranklin's Club,0.0,0.0,50.0,22.0,47,0.0,1.0
5,6,ArthurShepherd's Club,0.0,0.0,48.0,0.0,55,0.0,1.0
2,3,RhysNicholson's Club,0.0,0.0,47.0,1.0,53,0.0,1.0
6,7,FrancesPhillips's Club,0.0,0.0,41.0,12.0,44,0.0,1.0
4,5,CameronHarvey's Club,0.0,0.0,0.0,15.0,48,0.0,0.0
8,9,GeorgeBryan's Club,0.0,0.0,0.0,10.0,40,0.0,0.0
9,10,CarolKing's Club,0.0,0.0,0.0,3.0,55,0.0,0.0
0,1,SarahMatthews's Club,0.0,0.0,0.0,2.0,45,0.0,0.0


In [1135]:
# Order if online only

best_clubs_df = best_clubs_df.sort_values(['title_match_score', 'author_match_score', 'title_match_count', 'author_match_count', 'age_difference', 'user_count'], \
    axis=0, ascending = [False, False, False, False, True, False], kind='quicksort')
best_clubs_df

,club_id,club_name,location_match_score,title_match_score,author_match_score,age_difference,user_count,title_match_count,author_match_count
3,4,KarenQuinn's Club,0.0,44.0,44.0,3.0,50,1.0,1.0
7,8,SylviaFranklin's Club,0.0,0.0,50.0,22.0,47,0.0,1.0
5,6,ArthurShepherd's Club,0.0,0.0,48.0,0.0,55,0.0,1.0
2,3,RhysNicholson's Club,0.0,0.0,47.0,1.0,53,0.0,1.0
1,2,HelenGray's Club,100.0,0.0,41.0,4.0,63,0.0,1.0
6,7,FrancesPhillips's Club,0.0,0.0,41.0,12.0,44,0.0,1.0
0,1,SarahMatthews's Club,0.0,0.0,0.0,2.0,45,0.0,0.0
9,10,CarolKing's Club,0.0,0.0,0.0,3.0,55,0.0,0.0
8,9,GeorgeBryan's Club,0.0,0.0,0.0,10.0,40,0.0,0.0
4,5,CameronHarvey's Club,0.0,0.0,0.0,15.0,48,0.0,0.0
